# spaceship-titanic competition

In [ ]:
import polars as pl
import numpy as np
from sklearn.linear_model import LogisticRegression

In [122]:
# Kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

Could not find kaggle.json. Make sure it's located in /Users/yeyemac/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/


NameError: name 'exit' is not defined

## Read and Information of the DataSet

In [104]:
dataset_train = pl.read_csv('train.csv')
dataset_test = pl.read_csv('test.csv')

In [105]:
dataset_train.head()


PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
str,str,bool,str,str,f64,bool,f64,f64,f64,f64,f64,str,bool
"""0001_01""","""Europa""",false,"""B/0/P""","""TRAPPIST-1e""",39.0,false,0.0,0.0,0.0,0.0,0.0,"""Maham Ofracculy""",false
"""0002_01""","""Earth""",false,"""F/0/S""","""TRAPPIST-1e""",24.0,false,109.0,9.0,25.0,549.0,44.0,"""Juanna Vines""",true
"""0003_01""","""Europa""",false,"""A/0/S""","""TRAPPIST-1e""",58.0,true,43.0,3576.0,0.0,6715.0,49.0,"""Altark Susent""",false
"""0003_02""","""Europa""",false,"""A/0/S""","""TRAPPIST-1e""",33.0,false,0.0,1283.0,371.0,3329.0,193.0,"""Solam Susent""",false
"""0004_01""","""Earth""",false,"""F/1/S""","""TRAPPIST-1e""",16.0,false,303.0,70.0,151.0,565.0,2.0,"""Willy Santantines""",true


In [106]:
dataset_train.glimpse()

Rows: 8693
Columns: 14
$ PassengerId   <str> '0001_01', '0002_01', '0003_01', '0003_02', '0004_01', '0005_01', '0006_01', '0006_02', '0007_01', '0008_01'
$ HomePlanet    <str> 'Europa', 'Earth', 'Europa', 'Europa', 'Earth', 'Earth', 'Earth', 'Earth', 'Earth', 'Europa'
$ CryoSleep    <bool> False, False, False, False, False, False, False, True, False, True
$ Cabin         <str> 'B/0/P', 'F/0/S', 'A/0/S', 'A/0/S', 'F/1/S', 'F/0/P', 'F/2/S', 'G/0/S', 'F/3/S', 'B/1/P'
$ Destination   <str> 'TRAPPIST-1e', 'TRAPPIST-1e', 'TRAPPIST-1e', 'TRAPPIST-1e', 'TRAPPIST-1e', 'PSO J318.5-22', 'TRAPPIST-1e', 'TRAPPIST-1e', 'TRAPPIST-1e', '55 Cancri e'
$ Age           <f64> 39.0, 24.0, 58.0, 33.0, 16.0, 44.0, 26.0, 28.0, 35.0, 14.0
$ VIP          <bool> False, False, True, False, False, False, False, False, False, False
$ RoomService   <f64> 0.0, 109.0, 43.0, 0.0, 303.0, 0.0, 42.0, 0.0, 0.0, 0.0
$ FoodCourt     <f64> 0.0, 9.0, 3576.0, 1283.0, 70.0, 483.0, 1539.0, 0.0, 785.0, 0.0
$ ShoppingMall  <f64> 0.0

In [107]:
dataset_train.select(pl.col(pl.Float64)).describe()

statistic,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
str,f64,f64,f64,f64,f64,f64
"""count""",8514.0,8512.0,8510.0,8485.0,8510.0,8505.0
"""null_count""",179.0,181.0,183.0,208.0,183.0,188.0
"""mean""",28.82793,224.687617,458.077203,173.729169,311.138778,304.854791
"""std""",14.489021,666.717663,1611.48924,604.696458,1136.705535,1145.717189
"""min""",0.0,0.0,0.0,0.0,0.0,0.0
"""25%""",19.0,0.0,0.0,0.0,0.0,0.0
"""50%""",27.0,0.0,0.0,0.0,0.0,0.0
"""75%""",38.0,47.0,76.0,27.0,59.0,46.0
"""max""",79.0,14327.0,29813.0,23492.0,22408.0,24133.0


In [108]:
for col in dataset_train.select(pl.col(pl.Utf8)).columns:
    print(f"\n{'='*50}")
    print(f"Variable: {col}")
    print(f"{'='*50}")
    print(dataset_train[col].value_counts().sort('count', descending=True))


Variable: PassengerId
shape: (8_693, 2)
┌─────────────┬───────┐
│ PassengerId ┆ count │
│ ---         ┆ ---   │
│ str         ┆ u32   │
╞═════════════╪═══════╡
│ 0808_01     ┆ 1     │
│ 1150_01     ┆ 1     │
│ 4832_01     ┆ 1     │
│ 8467_01     ┆ 1     │
│ 0565_01     ┆ 1     │
│ …           ┆ …     │
│ 0779_01     ┆ 1     │
│ 5921_01     ┆ 1     │
│ 3707_01     ┆ 1     │
│ 4977_01     ┆ 1     │
│ 6066_02     ┆ 1     │
└─────────────┴───────┘

Variable: HomePlanet
shape: (4, 2)
┌────────────┬───────┐
│ HomePlanet ┆ count │
│ ---        ┆ ---   │
│ str        ┆ u32   │
╞════════════╪═══════╡
│ Earth      ┆ 4602  │
│ Europa     ┆ 2131  │
│ Mars       ┆ 1759  │
│ null       ┆ 201   │
└────────────┴───────┘

Variable: Cabin
shape: (6_561, 2)
┌──────────┬───────┐
│ Cabin    ┆ count │
│ ---      ┆ ---   │
│ str      ┆ u32   │
╞══════════╪═══════╡
│ null     ┆ 199   │
│ G/734/S  ┆ 8     │
│ B/82/S   ┆ 7     │
│ F/1194/P ┆ 7     │
│ G/1476/S ┆ 7     │
│ …        ┆ …     │
│ E/600/S  ┆ 1     

In [109]:
dataset_train.null_count()

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,201,217,199,182,179,203,181,183,208,183,188,200,0


In [110]:
dataset_train = dataset_train.drop('Name')
dataset_test = dataset_test.drop('Name')

## Nulls Strategy

In [111]:
n_filas = dataset_train.shape[0]
print(f"Registros: {n_filas}")

Registros: 8693


In [112]:
dataset_train = dataset_train.drop_nulls()
dataset_train.null_count()

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0


In [113]:
n_filas = dataset_train.shape[0]
print(f"Registros: {n_filas}")

Registros: 6764


## Encode

In [114]:
dataset_encoded = dataset_train.with_columns([
    # One-hot encoding para HomePlanet
    pl.col('HomePlanet').cast(pl.Categorical).to_physical().alias('HomePlanet_encoded'),
    # Lo mismo para otras categóricas
    pl.col('CryoSleep').cast(pl.Int8),  # bool → int
])

feature_cols = ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 
                'Spa', 'VRDeck', 'HomePlanet_encoded', 'CryoSleep']

## Model Training

### Logistic Regression

In [120]:

predictors = dataset_encoded.select(feature_cols)
outcome = 'Transported'

model = LogisticRegression(max_iter=1000)
model.fit(predictors.to_numpy(), 
          dataset_train[outcome].to_numpy().ravel())

print(f"intercept: {model.intercept_[0]:.3f}")
print(f"Coefficient primera feature: {model.coef_[0]}")
model

intercept: 0.383
Coefficient primera feature: [-4.72812872e-03 -1.11244873e-03  6.85073092e-04  7.56806157e-04
 -1.66533920e-03 -1.53660582e-03 -4.62260696e-01  1.68748965e+00]


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [118]:
y_true = dataset_encoded[outcome].cast(pl.Int8).to_numpy()  # 0/1
y_prob = model.predict_proba(predictors.to_numpy())[:, 1]   # Probabilidades
residuals = y_true - y_prob

print("Primeros 10 casos:")
print(f"Real:        {y_true[:10]}")
print(f"Predicho:    {y_prob[:10].round(3)}")
print(f"Residuales:  {residuals[:10].round(3)}")

Primeros 10 casos:
Real:        [0 1 0 0 1 1 1 1 1 1]
Predicho:    [0.55  0.219 0.    0.012 0.219 0.392 0.692 0.487 0.881 0.994]
Residuales:  [-0.55   0.781 -0.    -0.012  0.781  0.608  0.308  0.513  0.119  0.006]
